QKD - BB84 with Eve


In [1]:
%pip install cirq --quiet
import random
import cirq


Note: you may need to restart the kernel to use updated packages.


Phase 1 - Sending


In [2]:
print("Phase 1 - Sending")

no_of_qubits = int(input("Enter the no of Qubits: "))
alice_qubits = []
for _ in range(no_of_qubits):
    b = int(input("Enter every Qubit (0 or 1): "))
    if b in (0, 1):
        alice_qubits.append(b)
    else:
        raise ValueError("Only 0 or 1 allowed.")

print("Alice bits:", alice_qubits)

alice_basis = []
for _ in range(no_of_qubits):
    bb = input("Enter Alice basis (R or D): ").upper()
    if bb in ("R", "D"):
        alice_basis.append(bb)
    else:
        raise ValueError("Only R or D allowed.")

print("Alice bases:", alice_basis)

# Build qubits and Alice's circuit
qubits = [cirq.LineQubit(i) for i in range(no_of_qubits)]
circuit = cirq.Circuit()

# Bit preparation: 1 -> X
for i in range(no_of_qubits):
    if alice_qubits[i] == 1:
        circuit.append(cirq.X(qubits[i]))

# Basis: R -> no-op, D -> H
for i in range(no_of_qubits):
    if alice_basis[i] == "D":
        circuit.append(cirq.H(qubits[i]))


Phase 1 - Sending
Alice bits: [1, 0, 1, 0, 1]
Alice bases: ['D', 'R', 'D', 'R', 'D']


In [3]:
include_eve = input("Include Eve? (Y/N): ").strip().upper()
threshold_percent = float(input("Enter QBER threshold in percent (e.g., 11): "))

if include_eve == "Y":
    eve_basis = []
    for i in range(no_of_qubits):
        b = random.choice(["R", "D"])  # Eve random basis
        eve_basis.append(b)
        if b == "D":
            circuit.append(cirq.H(qubits[i]))
        circuit.append(cirq.measure(qubits[i], key=f"e{i}"))  # Eve measures
        if b == "D":
            circuit.append(cirq.H(qubits[i]))  # Re-send in D basis
    print("Eve bases:", eve_basis)
else:
    print("No Eve.")


Eve bases: ['R', 'D', 'D', 'R', 'R']


Phase 2 - Receiving


In [4]:
print("Phase 2 - Receiving")

bob_basis = []
for _ in range(no_of_qubits):
    bb = input("Enter Bob basis (R or D): ").upper()
    if bb in ("R", "D"):
        bob_basis.append(bb)
    else:
        raise ValueError("Only R or D allowed.")

# Bob rotation to Z: R -> no-op, D -> H
for i in range(no_of_qubits):
    if bob_basis[i] == "D":
        circuit.append(cirq.H(qubits[i]))

# Measure by Bob
circuit.append(cirq.measure(*qubits, key="bob"))

sim = cirq.Simulator()
res = sim.run(circuit, repetitions=1)
bob_bits = [int(res.measurements["bob"][0][i]) for i in range(no_of_qubits)]

print("Bob bases:", bob_basis)
print("Bob bits:", bob_bits)


Phase 2 - Receiving
Bob bases: ['D', 'D', 'D', 'D', 'D']
Bob bits: [0, 0, 1, 0, 0]


Phase 3 - Comparison and Eve detection


In [5]:

alice_key, bob_key = [], []
for i in range(no_of_qubits):
    if alice_basis[i] == bob_basis[i]:
        alice_key.append(alice_qubits[i])
        bob_key.append(bob_bits[i])

matched = len(alice_key)
errors = sum(1 for a, b in zip(alice_key, bob_key) if a != b)
qber = (errors / matched) * 100 if matched else 0.0

print("Matched:", matched)
print("Alice key:", alice_key)
print("Bob key:  ", bob_key)
print(f"QBER: {qber:.2f}%  |  Threshold: {threshold_percent:.2f}%")

if qber > threshold_percent:
    print("Eve Detected")
else:
    print("No Eve Detected")


Matched: 3
Alice key: [1, 1, 1]
Bob key:   [0, 1, 0]
QBER: 66.67%  |  Threshold: 11.00%
Eve Detected
